In [1]:
from dataclasses import replace
import numpy as np
from tradezoo.game import Game, Client, Constant, Trader
from tradezoo.market import Account, Market
from tradezoo.training import Experiment, LearningAgent, ReplayBuffer

In [2]:
num_traders = 2
trader_accounts = [Account(cash_balance=1, asset_balance=1) for _ in range(num_traders)]
client_accounts = [
    Account(cash_balance=float("inf"), asset_balance=float("inf"))
    for _ in range(num_traders)
]
traders = [
    Trader(
        agent=replace(
            LearningAgent.good_hyperparameters(),
            horizon=1,
            price_space=np.array(price_space),
            exploration_schedule=lambda step: min(1, 512 / (step + 256)),
            replay_buffer=ReplayBuffer.empty(capacity=256),
        ),
        account=trader_account,
        client=Client(
            account=client_account,
            for_account=trader_account,
            ask_process=Constant(ask),
            bid_process=Constant(bid),
        ),
    )
    for trader_account, client_account, ask, bid, price_space in zip(
        trader_accounts, client_accounts, [2, 8], [1, 4], [[2.1, 2.9], [3.1, 3.9]]
    )
]
game = Game.new(
    market=Market.from_accounts(trader_accounts + client_accounts),
    traders=traders,
)
experiment = Experiment.run_(game=game, num_steps=1024 * num_traders)


  0%|          | 0/2048 [00:00<?, ?it/s]

In [3]:
experiment.price_plot()

In [4]:
experiment.orders_plot(traders[0])

In [5]:
experiment.orders_plot(traders[1])